<a href="https://colab.research.google.com/github/LeoHanSung/SinChon8/blob/master/hansung/corona_week2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

hansung/corona_week2.ipynb


In [0]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [1]:
import pandas as pd
from pandas import Series
import numpy as np
from keras import optimizers
#from keras.models import Sequential, Model
#from keras.layers import Dense,Input
from keras import backend as K

from tensorflow.python.keras.layers import Input, Dense
from tensorflow.python.keras.models import Sequential, Model

Using TensorFlow backend.


ImportError: Traceback (most recent call last):
  File "C:\Users\leoha\anaconda3\lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "C:\Users\leoha\anaconda3\lib\site-packages\tensorflow\python\pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "C:\Users\leoha\anaconda3\lib\site-packages\tensorflow\python\pywrap_tensorflow_internal.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow_internal', fp, pathname, description)
  File "C:\Users\leoha\anaconda3\lib\imp.py", line 242, in load_module
    return load_dynamic(name, filename, file)
  File "C:\Users\leoha\anaconda3\lib\imp.py", line 342, in load_dynamic
    return _load(spec)
ImportError: DLL load failed: 지정된 모듈을 찾을 수 없습니다.


Failed to load the native TensorFlow runtime.

See https://www.tensorflow.org/install/errors

for some common reasons a

In [0]:
train_df = pd.read_csv('/kaggle/input/covid19-global-forecasting-week-2/train.csv')
train_df.head(5)

In [0]:
test_df = pd.read_csv('/kaggle/input/covid19-global-forecasting-week-2/test.csv')
test_df.head(5)

In [0]:
sub_df = pd.read_csv('/kaggle/input/covid19-global-forecasting-week-2/submission.csv')
sub_df.shape

In [0]:
print([train_df.columns, train_df.shape, 19404/66])
print([test_df.columns, test_df.shape],12642/43)

In [0]:
print("Country_Region")
print(len(train_df['Country_Region'].unique()), train_df['Country_Region'].unique())
print("Province_State")
print(len(train_df['Province_State'].unique()), train_df['Province_State'].unique())
print("Date")
print(len(train_df['Date'].unique()),train_df['Date'].unique())

In [0]:
print("Country_Region")
print(len(test_df['Country_Region'].unique()), test_df['Country_Region'].unique())
print("Province_State")
print(len(test_df['Province_State'].unique()), test_df['Province_State'].unique())
print("Date")
print(len(test_df['Date'].unique()),test_df['Date'].unique())

In [0]:
print("Date")
print(len(test_df['Date'].unique()),test_df['Date'].unique())

In [0]:
train_df['rownum'] = [i for i in range(1,67)] * 294

In [0]:
test_df['rownum'] = [i for i in range(58,101)] * 294

In [0]:
result_df = pd.DataFrame(columns=['ConfirmedCases','Fatalities'])
BATCH = 20
EPOCH = 3
HIDDENB_NUM_ONE = 256
OUTPUT_NUM = 2
INPUT_NUM = 1


for i in range(len(train_df['Country_Region'].unique())):
    if(pd.isnull(train_df[train_df['Country_Region']==train_df['Country_Region'].unique()[i]]['Province_State']).unique()[0]==True):
        nation_df = train_df[train_df['Country_Region']==train_df['Country_Region'].unique()[i]]
        #print("--------------------------nation_df-----------------------")
        #print(nation_df)
        x_train = nation_df['rownum']
        y_train = nation_df[['ConfirmedCases','Fatalities']]
        
        x_train_data = np.array(x_train)
        y_train_data = np.array(y_train)
        
        K.clear_session()
        model = Sequential()
        model.add(Dense(input_dim=INPUT_NUM, units=INPUT_NUM, activation="relu"))
        model.add(Dense(units=HIDDENB_NUM_ONE, activation="relu"))
        model.add(Dense(OUTPUT_NUM, activation="relu"))
        
        #print("nation:"+train_df['Country_Region'].unique()[i])
        
        model.compile(optimizer='adam', loss='mse')
        model.fit(x_train_data,y_train_data,batch_size=20, epochs=3, verbose=1)
        
        x_test = test_df[:43]['rownum']
        y_res = model.predict(x_test)
        
        #print("--------------------------y_res.shape-----------------------")
        #print(y_res.shape)
        result_df = pd.concat([result_df, pd.DataFrame({'ConfirmedCases': y_res[:, 0], 'Fatalities': y_res[:, 1]})])
        #print("--------------------------result_df.shape--------------------")
        print(result_df.shape)
        #print("---------------------------result_df----------------------")
        print(result_df)
        #print("#################################eon#######################")
    else:
        print(len(train_df[train_df['Country_Region']==train_df['Country_Region'].unique()[i]]['Province_State'].unique()))
        for m in range(len(train_df[train_df['Country_Region']==train_df['Country_Region'].unique()[i]]['Province_State'].unique())):
            
            province_df = train_df[train_df['Province_State']==train_df[train_df['Country_Region']==train_df['Country_Region'].unique()[i]]['Province_State'].unique()[m]]
            
            if(type(train_df[train_df['Country_Region']==train_df['Country_Region'].unique()[i]]['Province_State'].unique()[m])!=str):
                province_df = train_df[train_df['Country_Region']==train_df['Country_Region'].unique()[i]][train_df[train_df['Country_Region']==train_df['Country_Region'].unique()[i]]['Province_State'].isna()]
            
            #print("--------------------------province_df-----------------------")
            print(province_df)
            x_train = province_df['rownum']
            y_train = province_df[['ConfirmedCases','Fatalities']]

            x_train_data = np.array(x_train)
            y_train_data = np.array(y_train)

            K.clear_session()
            model = Sequential()
            model.add(Dense(input_dim=INPUT_NUM, units=INPUT_NUM, activation="relu"))
            model.add(Dense(units=HIDDENB_NUM_ONE, activation="relu"))
            model.add(Dense(OUTPUT_NUM, activation="relu"))
            
            #print(i)
            #print(m)
            #print(type(train_df[train_df['Country_Region']==train_df['Country_Region'].unique()[i]]['Province_State'].unique()[m]))
            #print(train_df['Country_Region'].unique()[i])
            #print(train_df[train_df['Country_Region']==train_df['Country_Region'].unique()[i]]['Province_State'].unique()[m])
            
            
            model.compile(optimizer='adam', loss='mse')
            model.fit(x_train_data,y_train_data,batch_size=30, epochs=3, verbose=1)

            x_test = test_df[:43]['rownum']
            y_res = model.predict(x_test)
            
            #print("--------------------------y_res.shape-----------------------")
            #print(y_res.shape)
            result_df = pd.concat([result_df, pd.DataFrame({'ConfirmedCases': y_res[:, 0], 'Fatalities': y_res[:, 1]})])
            #print("--------------------------result_df.shape--------------------")
            #print(result_df.shape)
            #print("---------------------------result_df----------------------")
            #print(result_df)
            #print("#################################eop#######################")

In [0]:
result_df.ConfirmedCases = result_df.ConfirmedCases.round(1)
result_df.Fatalities = result_df.Fatalities.round(1)
result_df[result_df['ConfirmedCases'] < 0] =0
result_df[result_df['Fatalities'] < 0] =0
result_df

In [0]:
sub = pd.DataFrame()
sub['ForecastId'] = np.array(test_df['ForecastId'])
sub['ConfirmedCases'] = np.array(result_df['ConfirmedCases'])
sub['Fatalities'] = np.array(result_df['Fatalities'])
sub.to_csv('submission.csv', index = False)